# First steps with o1

## Goal

Let's see if the new OpenAI's o1 model can write python code to solve the challenges.

## Imports

In [ ]:
import sys
import os
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from tqdm.auto import tqdm


# add path to python path
sys.path.append(os.path.realpath('../scripts/'))


from arc24.data import load_arc_data_with_solutions
from evaluation import plot_grids, plot_task, plot_grid

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (25, 4)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Load data

In [ ]:
train_data = load_arc_data_with_solutions('/mnt/hdd0/Kaggle/arc24/data/new_partitions/train_rs7.json')
val_data = load_arc_data_with_solutions('/mnt/hdd0/Kaggle/arc24/data/new_partitions/val_rs7.json')

## Experiments

In [ ]:
def create_prompt_for_o1(task):
    prompt = f"""Please write python code to implement the transformation shown in the examples below.
The examples are taken from the Abstraction and Reasoning challenge. Each example has an input and an output grid.
The size of the grids can vary from 1x1 to 30x30. The grids can contain up to 10 different colors. The colors are represented by integers from 0 to 9.
The transformations are always based on the following priors: objectness, goal-directed, numbers & counting, and basic geometry & topology.

{task['train']}
"""
    return prompt

In [ ]:
task_id = list(train_data.keys())[4]
print(task_id)
plot_task(train_data[task_id]); plt.suptitle(task_id); plt.show()
print(create_prompt_for_o1(train_data[task_id]))

- [00d62c1b](https://chatgpt.com/share/66e3f18b-3bd4-8012-9a39-8af9c706ab54), the description is correct, I have not checked the code
- [025d127b](https://chatgpt.com/share/66e3f269-0b08-8012-a0f2-1b6433301ed1) It has failed to understand the problem.
- [045e512c](https://chatgpt.com/share/66e3f3d5-bcb8-8012-9634-dbb226a04141), does not understand the task, hallucinates that it uses the code interpreter

In [ ]:
import numpy as np

def solve(input_grid):
    grid = np.array(input_grid)
    output_grid = grid.copy()
    visited = np.zeros_like(grid, dtype=bool)
    max_row, max_col = grid.shape

    # Get all unique colors except background (0)
    colors = [color for color in np.unique(grid) if color != 0]

    for color in colors:
        # Find positions of the current color
        positions = np.argwhere(grid == color)
        
        # Skip if already processed
        if visited[positions[0][0], positions[0][1]]:
            continue

        # Mark as visited
        for pos in positions:
            visited[pos[0], pos[1]] = True

        # Get bounding box
        min_row, min_col = positions.min(axis=0)
        max_row_obj, max_col_obj = positions.max(axis=0)

        # Extract the object
        object_grid = grid[min_row:max_row_obj+1, min_col:max_col_obj+1]
        object_mask = (object_grid == color).astype(int)

        # Pad to make it square
        rows, cols = object_mask.shape
        size = max(rows, cols)
        pad_rows = size - rows
        pad_cols = size - cols
        pad_top = pad_rows // 2
        pad_bottom = pad_rows - pad_top
        pad_left = pad_cols // 2
        pad_right = pad_cols - pad_left
        object_mask_padded = np.pad(object_mask, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')

        # Generate rotations
        rotated_masks = [object_mask_padded]
        rotated_masks.append(np.rot90(object_mask_padded, 1))
        rotated_masks.append(np.rot90(object_mask_padded, 2))
        rotated_masks.append(np.rot90(object_mask_padded, 3))

        # Merge rotations
        merged_mask = np.zeros_like(object_mask_padded)
        for mask in rotated_masks:
            merged_mask = np.logical_or(merged_mask, mask)

        # Remove padding
        merged_rows, merged_cols = merged_mask.shape
        unpad_top = pad_top
        unpad_bottom = merged_rows - pad_bottom
        unpad_left = pad_left
        unpad_right = merged_cols - pad_right
        merged_mask_unpadded = merged_mask[unpad_top:unpad_bottom, unpad_left:unpad_right]

        # Place the merged object back into the output grid
        out_rows = merged_mask_unpadded.shape[0]
        out_cols = merged_mask_unpadded.shape[1]
        mask_indices = np.where(merged_mask_unpadded)
        output_grid[min_row:min_row+out_rows, min_col:min_col+out_cols][mask_indices] = color

    return output_grid.tolist()


plot_grid(solve(train_data[task_id]['train'][1]['input'])); plt.show()

In [ ]:
def solve(task):
    import sys
    sys.setrecursionlimit(1000000)

    def flood_fill_from_edges(zero_grid, height, width):
        visited = [[False]*width for _ in range(height)]
        from collections import deque
        queue = deque()

        # Enqueue all edge zeros
        for x in range(width):
            if zero_grid[0][x] == 0:
                queue.append((0, x))
                visited[0][x] = True
            if zero_grid[height-1][x] == 0:
                queue.append((height-1, x))
                visited[height-1][x] = True
        for y in range(height):
            if zero_grid[y][0] == 0:
                queue.append((y, 0))
                visited[y][0] = True
            if zero_grid[y][width-1] == 0:
                queue.append((y, width-1))
                visited[y][width-1] = True

        # Flood fill zeros connected to edges
        while queue:
            y, x = queue.popleft()
            for dy, dx in [(-1,0),(1,0),(0,-1),(0,1)]:
                ny, nx = y + dy, x + dx
                if 0 <= ny < height and 0 <= nx < width:
                    if not visited[ny][nx] and zero_grid[ny][nx] == 0:
                        visited[ny][nx] = True
                        queue.append((ny, nx))
        return visited

    input_grid = task['input']
    height = len(input_grid)
    width = len(input_grid[0])

    # Create a grid where zeros are zeros, others are ones
    zero_grid = [[0 if cell == 0 else 1 for cell in row] for row in input_grid]

    # Flood fill from edges to mark zeros connected to the edge
    connected_to_edge = flood_fill_from_edges(zero_grid, height, width)

    # Now, zeros not connected to edge are enclosed zeros
    output_grid = [row[:] for row in input_grid]  # Deep copy of input grid

    for y in range(height):
        for x in range(width):
            if zero_grid[y][x] == 0 and not connected_to_edge[y][x]:
                # Enclosed zero, replace with 4
                output_grid[y][x] = 4

    return output_grid
plot_grid(solve(train_data['00d62c1b']['train'][2])); plt.show()